Імпорт необхідних бібліотек

hashlib - використовується для роботи з функцією хешування алгоритмом sha256

re - використовується для пошуку кількості нулів у префіксі хешу

time - використовується для визначення значення параметру timestamp у блоці

json - використовується для створення більш читабельного формату виведення інформації про блок на екран

In [1]:
import hashlib
import re
import time
import json

Допоміжна функція для пошуку кількості нулів у префіксі хешу. Використовує регулярний вираз і повертає строку з знайденним префіксом або пусту, якщо нулі відсутні

In [2]:
def getZeros(string):
    result = re.search(r"^0+", string)
    if result:
        return result.group(0)
    else:
        return ""

Клас BlockHeader, має функцію майнінгу та створення власного хешу. У конструкторі використовує аргумент хешу попереднього блока для запису у параметр prevBlockHash. Також створює timestamp та початковий хеш блоку. Хеш блоку перезаписується у процесі майнінгу. Функція майнінгу має задану умовою складність 4, працює з власним хешом блоку, що був створений на етапі ініціалізації інстансу класу і у процесі перебору nonce перезаписує хеш. Функція створення хешу використовує дані про блок для створення строки, що використовується для хешування. Хешування проводиться у два раунди алгоритмом sha256, що взято з бібліотеки hashlib

In [3]:
class BlockHeader:
    version = 1
    merkleRoot = ""
    timestamp = 0
    bits = "ffff001f"
    nonce = 0
    blockHash = ""
    prevBlockHash = ""
    
    def __init__(self, prevBlockHash):
        self.prevBlockHash = prevBlockHash
        self.timestamp = int(time.time())
        self.blockHash = self.makeHash()

    def mine(self):
        difficultyTarget = 4
        while len(getZeros(self.blockHash)) != difficultyTarget:
            self.nonce += 1
            self.blockHash = self.makeHash()
        

    def makeHash(self):
        preEncodeString = f"{self.version}{self.prevBlockHash}{self.merkleRoot}{self.timestamp}{self.bits}{self.nonce}"
        hashRoundOne = hashlib.sha256(preEncodeString.encode('utf-8')).hexdigest()
        hashRoundTwo = hashlib.sha256(hashRoundOne.encode('utf-8')).hexdigest()
        return str(hashRoundTwo)


Основний клас Block. У конструкторі використовує хеш попереднього блоку і на його основі створює інстанс хедеру, також записує висоту. Містить функцію майнінгу, що викликає майнінг хедеру, створює транзакцію, змінює лічильник транзакцій та записує хеш транзакції у параметр merkleRoot хедеру. Функція повернення інформації про блок, що буде використовуватись для отримання висоти та хешу блоку при створенні наступного блоку. Також створена допоміжна функція toJSON, яка використовується для створення більш читабельного формату виводу інформації на екран

In [4]:
class Block:
    blockHeader = ""
    blockSize = 1
    txCount = 0
    txs = ""
    height = 0
    
    def __init__(self, height, prevBlockHash):
        self.blockHeader = BlockHeader(prevBlockHash)
        self.height = height

    def mine(self):
        self.blockHeader.mine();
        self.txs = f"Andrii sent {self.height} coins to Alice"
        self.txCount += 1
        self.blockHeader.merkleRoot = str(hashlib.sha256(self.txs.encode('utf-8')).hexdigest())
        return self
    
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, sort_keys=True, indent=4)
    
    def getBlockData(self):
        print(self.toJSON())
        return self

Клас Blockchain. Містить масив блоків chain, функції створення стартового блоку, для якого хеш попереднього блоку є нульовою строкою, та функцію створення нових блоків з їх додаванням до масиву. Нові блоки створюються з посиланням на попередні: для цього використовується висота попереднього блоку та його хеш. Нові блоки майняться та зберігаються до масиву chain

In [5]:
class Blockchain:
    chain = []

    def genesisBlock(self):
        zeroHash = "".rjust(64, "0")
        initialBlock = Block(0, zeroHash).mine().getBlockData()
        self.chain.append(initialBlock)
        return self

    def addBlock(self):
        previousBlock = self.chain[len(self.chain) - 1];
        newBlock = Block(previousBlock.height + 1, previousBlock.blockHeader.blockHash).mine().getBlockData();

        self.chain.append(newBlock);
        return self

Безпосереднє використання коду. Створення інстансу класу Blockchain, за його допомогою створення стартового блоку та додавання двох нових блоків. У процесі на екран виводиться інформація про кожний блок

In [6]:
blockchainInstance = Blockchain()
genesisBlock = blockchainInstance.genesisBlock()
genesisBlock.addBlock()
genesisBlock.addBlock()

{
    "blockHeader": {
        "blockHash": "000048ac691b6f9459b764af8236fb8f9a740cfe74e7dd31564b839f1c8374d3",
        "merkleRoot": "440c15781fe1fd99a6b57d72d34ffb21d10a629e5e9dd69d1fe34bcd9f21affd",
        "nonce": 4623,
        "prevBlockHash": "0000000000000000000000000000000000000000000000000000000000000000",
        "timestamp": 1670187805
    },
    "height": 0,
    "txCount": 1,
    "txs": "Andrii sent 0 coins to Alice"
}
{
    "blockHeader": {
        "blockHash": "0000e2fdbbedb1dc05b38bf65fde3f12fd4c91f1e110668c71ba60a3531bf894",
        "merkleRoot": "550394aef6271f3cfb36e5b25f0bbff2fd2f8369522b9853518573c9a910e189",
        "nonce": 354328,
        "prevBlockHash": "000048ac691b6f9459b764af8236fb8f9a740cfe74e7dd31564b839f1c8374d3",
        "timestamp": 1670187805
    },
    "height": 1,
    "txCount": 1,
    "txs": "Andrii sent 1 coins to Alice"
}
{
    "blockHeader": {
        "blockHash": "0000d9957f79f516c1a95a083a71ea83c30f28fa6cbf0b802b12e2495a18d781",
        "merkl